### Object Detection with ssd

In [1]:
### Importing necessary libraries
import torch
from torch.autograd import Variable
import cv2
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio

In [2]:
### Defining a function that will do the detection
def detect(frame, net, transform):
    height, width = frame.shape[:2]  ### get the height and width from the image
    frame_t = transform(frame)[0] ### get the first variable, thus the transformed frame
    x = torch.from_numpy(frame_t).permute(2,0,1) ### converting to torch tensor and right color, ref RGB to BRG
    x = x.unsqueeze(0)
    with torch.no_grad():
        y = net(x)
    
    detections = y.data
    scale = torch.Tensor([width, height, width, height])
    ## elements of the detection tensor; [batch, number of classes(objects to be detected),
    ### number of occurence of the class, tuple of five elements; (score, x0, Y0, x1, Y1)]
    
    for i in range(detections.size(1)):
        j = 0
        while detections[0,i,j,0]>=0.6:
            pt = (detections[0,i,j, 1:] * scale).numpy()
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255,0,0), 2)
            cv2.putText(frame, labelmap[i-1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            j += 1
    return frame

In [3]:
#### Creating the SSD Neural networ with pretrained weight, 'ssd300_mAP_77.43_v2.pth'
net = build_ssd('test')  ### neural network created 
net.load_state_dict(torch.load('ssd300_mAP_77.43_v2.pth', map_location = lambda storage, loc: storage))

### creating the transformation
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0))

In [4]:
reader = imageio.get_reader('class_colleagues.mp4')
fps = reader.get_meta_data()['fps']
writer = imageio.get_writer('output1.mp4', fps = fps)
for i, frame in enumerate(reader):
    frame = detect(frame, net.eval(), transform)
    writer.append_data(frame)
    print(i)
#### close the process
writer.close()

The frame size for reading (848, 480) is different from the source frame size (480, 848).


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
